# Example notebook to search geometry using suxarray

Note that this is based on `uxarray` master version (or v2023.04.1).

In [ ]:
import shapely
from shapely.geometry import Point
import schismviz.suxarray as sx

## Read a gr3 file to a grid

In [ ]:
path_testgrid_gr3 = '../tests/testdata/testmesh.gr3'
grid = sx.read_hgrid_gr3(path_testgrid_gr3)

## Find an horizonal grid element from a x-, and y-coordinate

The return value is a list of elements that intersects with a point from the given coordinate. The values are __zero-based__ indices.

In [ ]:
# The experted answer is 123.
grid.find_element_at(0.5, 0.3)

### Same example as above but using a STRtree and a Shapely point directly

In [ ]:
point = Point(0.5, 0.3)
elem_i = grid.elem_strtree.query(point, predicate='intersects')

## Calculate distances from a point to the nodes of an element found above

## Get the node indices of the element found above

In [ ]:
nodes_in_elem = grid.Mesh2_face_nodes.isel(nSCHISM_hgrid_face=elem_i[0])

### Get the number of nodes in an element

In [ ]:
n_nodes_in_elem = grid.Mesh2_face_dimension.isel(nMesh2_face=elem_i[0]).values[()]
n_nodes_in_elem

### Select node indices for the element

Note that these are one-based (for now).

In [ ]:
xs = grid.Mesh2_node_x.isel(nSCHISM_hgrid_node=nodes_in_elem[:n_nodes_in_elem]).values
ys = grid.Mesh2_node_y.isel(nSCHISM_hgrid_node=nodes_in_elem[:n_nodes_in_elem]).values

In [ ]:
dist = [shapely.distance(Point(x, y), point) for x, y in zip(xs, ys)]
dist